In [5]:
import os
import json
import joblib
import pandas as pd
hotpot = joblib.load('hotpot-qa-distractor-sample.joblib').reset_index(drop = True)
# print(f'{hotpot =}\n{type(hotpot)=}')
# print(hotpot.iloc[0])
# hotpot.to_csv('hotpot-qa-distractor-sample.csv')
hotpot


,id,question,answer,type,level,supporting_facts,context
0,5a7613c15542994ccc9186bf,VIVA Media AG changed it's name in 2004. What ...,Gesellschaft mit beschränkter Haftung,bridge,hard,"{'title': ['VIVA Media', 'Gesellschaft mit bes...","{'title': ['Constantin Medien', 'VIVA Poland',..."
1,5adf2fa35542993344016c11,Which of Jonny Craig and Pete Doherty has been...,"Jonny"" Craig",comparison,hard,"{'title': ['Jonny Craig', 'Jonny Craig', 'Pete...","{'title': ['Pete Doherty', 'Relativity (Emaros..."
2,5adfdef9554299025d62a36b,Where was the first governor after the The Mis...,"Bath, Maine",bridge,hard,"{'title': ['Maine gubernatorial election, 1820...","{'title': ['Compromise of 1790', 'Anti-Nebrask..."
3,5a7180205542994082a3e856,"The creator of ""Wallace and Gromit"" also creat...",Creature Comforts,bridge,hard,"{'title': ['Creature Comforts', 'Creature Comf...","{'title': ['Creature Comforts', 'Tata Steel Zo..."
4,5a78bc6b554299148911f979,Woman's Era and Naj are what kind of magazines?,fortnightly women interest magazine,comparison,hard,"{'title': ['Woman's Era', 'Naj'], 'sent_id': [...","{'title': ['Lifestyle trends and media', 'Chin..."
...,...,...,...,...,...,...,...
95,5ac509e95542994611c8b333,The chicken is a type of dance pattern that is...,the Twist,bridge,hard,"{'title': ['Chicken (dance)', 'Chicken (dance)...","{'title': ['Dance move', 'Decoded neurofeedbac..."
96,5a8b42be55429949d91db515,What profession does John Lanchester and Alan ...,novelist,comparison,hard,"{'title': ['John Lanchester', 'Alan Dean Foste...","{'title': ['Bloodhype', 'Alan Dean Foster', 'T..."
97,5ab9b2c7554299743d22ebaf,Are both Lygodium or Maxillaria a genus of orc...,no,comparison,hard,"{'title': ['Lygodium', 'Maxillaria'], 'sent_id...","{'title': ['Paracaleana', 'Heterotaxis', 'Orni..."
98,5a7b63eb55429931da12ca7e,What city does Paul Clyne and David Soares hav...,New York,bridge,hard,"{'title': ['Paul Clyne', 'David Soares'], 'sen...","{'title': ['Rhabdodontidae', 'Paul Clyne', 'Tu..."


In [13]:
med = []
with open('0604_original_test.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line.strip())
        med.append(record)
med = pd.DataFrame(med)
med

,question,answer,options,meta_info,answer_idx,evidence
0,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者游文胜，男，50岁，主因“乙肝表面抗...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
1,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者孙国庆，男，61岁，主因“间断腹痛腹...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
2,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者郭佃峰，男，主因“巩膜黄染，确诊原发...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
3,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者30余年前体检发现肝功能异常，ALT...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
4,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者6天前（2021-11-24）因“发...,原发性肝癌,"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[C],NaN
...,...,...,...,...,...,...
195,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者中年女性，慢性病程，急性发作；9年前...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。...
196,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者老年女性，慢性病程；13年前体检发现...,"[脂肪肝, 自身免疫性肝炎]","{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,"[A, E]","老年女性患者，慢性病程，体检发现肝功能异常,以转氨酶轻度升高为主，IgG、抗核抗体升高，腹部..."
197,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者主因原位肝移植术后5年入院。入院查体...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],1.该患者为中年男性，慢性病程\n2.原位肝移植术后5年\n3.既往：高血压4年余，血压最高...
198,请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者主因“反复呕血、黑便3年余，TIPS...,[脂肪肝],"{'A': '脂肪肝', 'B': '药物性肝损伤', 'C': '原发性肝癌', 'D':...",肝病,[A],患者青年男性，慢性病程；临床以消化道出血起病，进一步胃镜、影像学检查发现肝硬化合并食管胃底静...


In [ ]:
# for ind, row in hotpot.iterrows():
#     supporting_articles = row['supporting_facts']['title']
#     articles = row['context']['title']
#     sentences = row['context']['sentences'] 
#     supporting_paragraphs = []
#     for article in supporting_articles:
#         supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
#         supporting_paragraphs.append(supporting_paragraph)
#     supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
#     hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

In [14]:
def process_answer(answer):
    if isinstance(answer,str): return answer
    elif isinstance(answer,list):
        if len(answer)==1: return answer[0]
        else: return '+'.join(answer)
    else: print(answer)
prompt =  "请使用你的已有知识，判断该病人是以下这7个病中的哪一种或多种疾病。脂肪肝，药物性肝损伤，原发性肝癌，酒精性肝病，自身免疫性肝炎，肝囊肿，肝血管瘤。如果病人是多种疾病，用'+'号连接。以下回答都是正确格式：脂肪肝。脂肪肝+药物性肝损伤。药物性肝损伤+脂肪肝+自身免疫性肝炎。"
for ind, row in med.iterrows():
    raw_q = row['question']
    context = raw_q.replace('请判断该病人是以下哪种疾病。病人检查如下：','') # 反整理出病情context
    med.at[ind,'context'] = context 
    # TODO:对齐195，199的evidence的提取
    med.at[ind,'QAonly'] = prompt+'||'+context
    med.at[ind,'process_answer'] = process_answer(row['answer'])
    # TODO: 如果用context模式，要设计COT引导的sample（看代码）
    

KeyboardInterrupt: 

In [16]:
def process_answer(answer):
    if isinstance(answer,str): return answer
    elif isinstance(answer,list):
        if len(answer)==1: return answer[0]
        else: return '+'.join(answer)
    else: print(answer)
med['answer'].apply(process_answer).unique()

array(['原发性肝癌', '肝囊肿', '肝囊肿+脂肪肝', '肝血管瘤+肝囊肿', '肝血管瘤',
       '原发性肝癌+脂肪肝+肝血管瘤+肝囊肿', '脂肪肝+肝血管瘤', '药物性肝损伤+酒精性肝病+脂肪肝', '酒精性肝病+肝血管瘤',
       '酒精性肝病+脂肪肝', '酒精性肝病', '自身免疫性肝炎', '自身免疫性肝炎+肝囊肿', '脂肪肝+自身免疫性肝炎',
       '脂肪肝+自身免疫性肝炎+药物性肝损伤', '肝血管瘤+自身免疫性肝炎', '肝血管瘤+自身免疫性肝炎+药物性肝损伤',
       '自身免疫性肝炎+药物性肝损伤', '肝囊肿+自身免疫性肝炎', '肝囊肿+自身免疫性肝炎+药物性肝损伤', '药物性肝损伤',
       '肝囊肿+脂肪肝+药物性肝损伤', '肝囊肿+药物性肝损伤', '酒精性肝病+药物性肝损伤', '脂肪肝+药物性肝损伤',
       '自身免疫性肝炎+脂肪肝+药物性肝损伤', '脂肪肝', '脂肪肝+肝囊肿', '药物性肝损伤+脂肪肝',
       '药物性肝损伤+脂肪肝+自身免疫性肝炎'], dtype=object)

In [11]:
data.loc[1,'options']

{'A': '脂肪肝',
 'B': '药物性肝损伤',
 'C': '原发性肝癌',
 'D': '酒精性肝病',
 'E': '自身免疫性肝炎',
 'F': '肝囊肿',
 'G': '肝血管瘤'}

In [ ]:
请使用你的已有知识，判断该病人是以下这7个病中的哪一种或多种疾病。脂肪肝，药物性肝损伤，原发性肝癌，酒精性肝病，自身免疫性肝炎，肝囊肿，肝血管瘤。
如果病人是多种疾病，用'和'字连接。以下回答都是正确格式：脂肪肝。脂肪肝和药物性肝损伤。脂肪肝和自身免疫性肝炎。


In [8]:
data.loc[195,'question']

'请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。腹部超声提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动。2年前于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前患者体检发现GGT 710U/L。1月前患者复查示GGT 496U/L。 主诉：发现肝功能异常9年，加重3月。 现病史：9年前，体检发现肝功能异常，生化示ALT 62U/L，AST 36U/L，ALP 93U/L,GGT 237U/L,TG 2.36mmol/l，TBIL 15.1umol/L，DBIL 3.8umol/L，TP 73g/L，ALB 43g/L。患者自感乏力、多睡，肝区偶有隐痛，无恶心、呕吐、黄疸、发热、关节疼痛等不适，于北京协和医院查彩超提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动,多次查彩超均提示脂肪肝，查乙肝、丙肝、戊肝标志物阴性，自身免疫性肝病相关检查阴性，未曾行肝脏核磁或CT检查。2年前，于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前，患者体检发现GGT 710U/L，继续口服易善复治疗。1月前，患者复查示GGT 496U/L。现患者为求进一步诊治，收入我科。患者平素精神、睡眠可，食欲可，大小便无异常。体重近期无明显变化。 既往史: “特纳综合征”史35年。“骨质疏松”史20余年，给予钙片、阿仑磷酸钠片口服治疗。“糖尿病”史11年，空腹血糖12mmol/L，餐后血糖20mmol/L，现“诺和锐早15u中15u晚15u ih”控制血糖，血糖控制可。“甲状腺功能减退”史5年，现口服“左甲状腺素钠片 1粒 qd”，2019.10.24查FT3 FT4 TSH3正常。否认高血压、心脏病史，否认脑血管病、精神疾病史。否认肝炎史、结核史、疟疾史。否认手术、外伤、输血史，否认药物过敏史，有山药皮过敏史，预防接种史不详。其他系统回顾无特殊。 个人史：出生并久居于本地，否认

In [10]:
data.loc[195,'evidence']

'患者中年女性，慢性病程，急性发作；9年前体检发现肝功能异常，转氨酶轻度升高，转肽酶升高明显。腹部超声提示脂肪肝，给予“易善复”口服治疗，肝功能恢复正常，但停药后反复。后规律复查示GGT在300-450U/L、ALT在40-100U/L、AST在30-70U/L、ALP在100-160U/L左右波动。2年前于我院行肝穿刺检查示脂肪肝。出院后给予“易善复”规律治疗。3月前患者体检发现GGT 710U/L。1月前患者复查示GGT 496U/L。根据患者症状、体征、既往史、辅助检查等，患者非酒精性脂肪型肝炎、2型糖尿病、骨质疏松、特纳综合征、甲状腺功能减低诊断明确。'

In [9]:
data.loc[1,'question']

'请判断该病人是以下哪种疾病。病人检查如下：入院情况：患者孙国庆，男，61岁，主因“间断腹痛腹胀1月余”于2021-04-01入院。 主诉：间断腹痛腹胀1月余 现病史：患者1月前无明显诱因出现腹痛，腹胀，疼痛性质不明，无恶心、呕吐，腹泻，黄疸等，无背部放射痛，无便血、黑便，无大便发白，无小便颜色加深，无反复发作的低血糖症状，遂就诊于我院门诊行腹部增强CT提示：1、肝脏多发异常密度影，考虑多发转移瘤；2、胰腺尾部低密度影，考虑胰腺癌，伴脾动静脉受侵可能，脾门处侧枝循环形成；3、大网膜、肠系膜及腹膜索条及结节软组织密度影，种植转移可能；4、左侧肾上腺异常改变，转移不除外；5、肠系膜、腹膜后多发淋巴结，建议动态观察；6、双肾多发小囊肿；7、慢性胆囊炎可能；8、左侧精囊腺钙化灶；9、盲肠、升结肠多发小憩室。患者为行进一步治疗，就诊于我院门诊，以“胰腺占位性病变”收入我科，自发病以来，神清好，精神尚可，睡眠尚可，饮食好，大小便正常，体重无明显变化。 既往史: 史：否认高血压、心脏病史，否认糖尿病、脑血管病、精神疾病史。否认肝炎史、结核史、疟疾史。手术史：无，过敏史：无，输血史：无，预防接种史：无，传染病史：无。其他系统回顾无特殊。 个人史：出生并久居于本地，否认疫水、疫区接触史，否认其他放射性物质及毒物接触史。免疫接种史不详。有吸烟史，吸烟30年，5-6支/天，已戒酒1月。否认去过武汉、否认接触过从武汉来的人。 家族史：nan 腹部彩色超声检查-检查所见:肝大小正常，实质回声：前部增强，底部减弱。肝内见多发低回声结节，大者位于S2，大小约2.8x1.6cm，边界清，尚规则。肝内血管走行正常，肝内胆管未见扩张。门静脉不扩张。  胆囊大小正常，壁不厚，光滑，胆囊腔内未见异常回声，胆总管未见扩张。  胰腺大小正常，回声欠均匀，胰管未见扩张。胰尾部见低回声实性结节，大小约4.4x3.2cm，边界欠清，欠规则。  脾厚约4.8cm，长约10.3cm，脾实质回声均匀，脾门处脾静脉受压。  双肾大小正常，回声正常，血流信号丰富。双侧肾盂、肾盏未见扩张。 腹部彩色超声检查结论:胰尾部实性占位，倾向Ca  肝内多发低回声结节，倾向M  脾大  脂肪肝腹部核磁共振检查-腹部核磁检查所见:胰腺尾部见一类圆形状异常信号影，大小约3.2cm×5.4cm×3.7cm（前后径×左右径×上下径），信号欠均匀